In [ ]:
#!pip install azureml-tensorboard

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="a0a6d112-ab35-410a-b961-1280cf758bfc")
ws = Workspace.from_config(auth=interactive_auth)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Set experiment name and start the run

In [ ]:
experiment_name = 'export-to-tensorboard'
exp = Experiment(ws, experiment_name)
root_run = exp.start_logging()

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y=True)

columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
data = {
    "train":{"x":x_train, "y":y_train},        
    "test":{"x":x_test, "y":y_test}
}

In [ ]:
# Example experiment
from tqdm import tqdm

alphas = [.1, .2, .3, .4, .5, .6 , .7]

# try a bunch of alpha values in a Linear Regression (Ridge) model
for alpha in tqdm(alphas):
    # create a bunch of child runs
    with root_run.child_run("alpha" + str(alpha)) as run:
        reg = Ridge(alpha=alpha)
        reg.fit(data["train"]["x"], data["train"]["y"])
        
        preds = reg.predict(data["test"]["x"])
        mse = mean_squared_error(preds, data["test"]["y"])
        # End train and eval

        # log alpha, mean_squared_error and feature names in run history
        root_run.log("alpha", alpha)
        root_run.log("mse", mse)

## Export Run History to Tensorboard logs

In [ ]:
# Export Run History to Tensorboard logs
from azureml.tensorboard.export import export_to_tensorboard
import os

logdir = 'exportedTBlogs'
log_path = os.path.join(os.getcwd(), logdir)
try:
    os.stat(log_path)
except os.error:
    os.mkdir(log_path)
print(logdir)

# export run history for the project
export_to_tensorboard(root_run, logdir)

# or export a particular run
# export_to_tensorboard(run, logdir)

In [ ]:
root_run.complete()

## Start Tensorboard

Or you can start the Tensorboard outside this notebook to view the result

In [ ]:
from azureml.tensorboard import Tensorboard

# The Tensorboard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard([], local_root=logdir, port=6006)

# If successful, start() returns a string with the URI of the instance.
tb.start()

## Stop Tensorboard

When you're done, make sure to call the `stop()` method of the Tensorboard object.

In [ ]:
tb.stop()